# <center>Anomaly Detection in Surveillance videos</center>


# Preprocessing

Extracting out the category labels

In [18]:
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import pickle
from tqdm import tqdm
import numpy as np
import random

test_dir = '/kaggle/input/crimedataset/Test/Test'
train_dir = '/kaggle/input/crimedataset/Train/Train'

# Define the categories and labels
categories_labels = {
    'Fighting': 0, 
    'Shoplifting': 1, 
    'Abuse': 2, 
    'Arrest': 3, 
    'Shooting': 4, 
    'Robbery': 5, 
    'Explosion': 6,
    'RoadAccidents':7
}



def load_data(base_dir, categories_labels):
    data = []
    
    # Go through each category
    for category, label in categories_labels.items():
        # The path to the category directory
        category_dir = os.path.join(base_dir, category)

        # Make sure the directory exists
        if os.path.isdir(category_dir):
            # Go through each file in the directory
            for filename in tqdm(os.listdir(category_dir), desc=f"Loading {category}"):
                # Make sure the file is an image
                if filename.endswith(".jpg") or filename.endswith(".png"):
                    # The path to the image
                    image_path = os.path.join(category_dir, filename)

                    try:
                        # Load the image
                        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

                        # Resize the image
                        image = cv2.resize(image, (50, 50))

                        # Reshape the image to 4D array 
                        #(ImageDataGenerator requires 4D array)
                        image = image.reshape((1,) + image.shape + (1,))

                        # Add the image and its label to the data
                        data.append([image, label])
                    except Exception as e:
                        print(f"Error loading image {image_path}: {e}")

    return data

# Load the training and test data
training_data = load_data(train_dir, categories_labels)
test_data = load_data(test_dir, categories_labels)

# Combine the training and test data
total_data = training_data + test_data

print(f"Loaded {len(total_data)} images.")


Loading RoadAccidents: 100%|██████████| 1857/1857 [00:04<00:00, 441.22it/s]

Loaded 11356 images.


In [19]:
print(len(training_data))
print(len(test_data))
print(len(total_data))

5678
5678
11356


# Model

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers import LSTM, TimeDistributed, Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Flatten
from keras.layers import concatenate
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import time


# Initialize lists to store the images and the labels
images = []
labels = []

# Go through each image and its label in the total_data
for image, label in total_data:
    images.append(image)
    labels.append(label)

# Convert the lists into numpy arrays
images = np.array(images)
labels = np.array(labels)
print(images.shape)

# Reshape images for LSTM
images_lstm = images.reshape(images.shape[0], -1, 1)  # Added third dimension for features

# Set a seed for reproducibility
seed = 51 #9 + 14 + 28

# Split the data into training and testing sets for CNN
train_images_cnn, test_images_cnn, train_labels_cnn, test_labels_cnn = train_test_split(images, labels, test_size=0.1, random_state=seed)

# Split the data into training and testing sets for LSTM
train_images_lstm, test_images_lstm, train_labels_lstm, test_labels_lstm = train_test_split(images_lstm, labels, test_size=0.1, random_state=seed)

# Convert labels to categorical for CNN
train_labels_cnn = np_utils.to_categorical(train_labels_cnn, len(categories_labels))
test_labels_cnn = np_utils.to_categorical(test_labels_cnn, len(categories_labels))

# Convert labels to categorical for LSTM
train_labels_lstm = np_utils.to_categorical(train_labels_lstm, len(categories_labels))
test_labels_lstm = np_utils.to_categorical(test_labels_lstm, len(categories_labels))

# Remove the second dimension from your data
train_images_cnn = np.squeeze(train_images_cnn, axis=1)
test_images_cnn = np.squeeze(test_images_cnn, axis=1)



(11356, 1, 50, 50, 1)


# Custom Model - CNN & LSTM

In [21]:
model_CNN = Sequential()
model_CNN.add(Conv2D(64, kernel_size=(3, 3), padding='same', input_shape=(50, 50, 1)))
model_CNN.add(LeakyReLU(alpha=0.1)) 
model_CNN.add(MaxPooling2D((2, 2), padding='same')) 
model_CNN.add(Dropout(0.25))
model_CNN.add(Conv2D(128, (3, 3), padding='same')) 
model_CNN.add(LeakyReLU(alpha=0.1)) 
model_CNN.add(MaxPooling2D(pool_size=(2, 2), padding='same')) 
model_CNN.add(Dropout(0.25))
model_CNN.add(Conv2D(256, (3, 3), padding='same'))
model_CNN.add(LeakyReLU(alpha=0.1))
model_CNN.add(MaxPooling2D(pool_size=(2, 2), padding='same')) 
model_CNN.add(Dropout(0.4))  
model_CNN.add(Flatten()) 
model_CNN.add(Dense(256)) 
model_CNN.add(LeakyReLU(alpha=0.1))            
model_CNN.add(Dropout(0.5)) 

# LSTM Model
model_lstm = Sequential()
model_lstm.add(LSTM(units = 8, return_sequences = True, input_shape = (2500, 1), activation='tanh'))
model_lstm.add(LSTM(units = 8, return_sequences = True))
model_lstm.add(Dense(4, activation='tanh'))
model_lstm.add(Dropout(0.2))
model_lstm.add(Flatten())

# Combine CNN and LSTM model
nb_classes = 7
combined = concatenate([model_CNN.output, model_lstm.output], axis=-1)
output = Dense(nb_classes, activation='softmax')(combined)
model_final = Model(inputs=[model_CNN.input, model_lstm.input], outputs=output)

# Plot and compile the model
plot_model(model_final, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

model_final.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks
csv_logger = CSVLogger('training.log', separator=',', append=False)
mc = ModelCheckpoint('CNN_LSTM.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

# Training
time1 = time.time()
history = model_final.fit([train_images_cnn, train_images_lstm], train_labels_lstm, batch_size=1000, epochs=20, validation_data=([test_images_cnn, test_images_lstm], test_labels_lstm), callbacks=[mc, csv_logger])
print (("Training time=", time.time()-time1))

# Save training history
np.save("CNN_LSTM_history.npy", history.history)

<!-- #CNN Model
model_CNN = Sequential()
model_CNN.add(Conv2D(64, kernel_size=(3, 3), padding='same', input_shape=(50, 50, 1)))
model_CNN.add(LeakyReLU(alpha=0.1)) 
model_CNN.add(MaxPooling2D((2, 2), padding='same')) 
model_CNN.add(Dropout(0.25))
model_CNN.add(Conv2D(128, (3, 3), padding='same')) 
model_CNN.add(LeakyReLU(alpha=0.1)) 
model_CNN.add(MaxPooling2D(pool_size=(2, 2), padding='same')) 
model_CNN.add(Dropout(0.25))
model_CNN.add(Conv2D(256, (3, 3), padding='same'))
model_CNN.add(LeakyReLU(alpha=0.1))
model_CNN.add(MaxPooling2D(pool_size=(2, 2), padding='same')) 
model_CNN.add(Dropout(0.4))  
model_CNN.add(Flatten()) 
model_CNN.add(Dense(256)) 
model_CNN.add(LeakyReLU(alpha=0.1))            
model_CNN.add(Dropout(0.5)) 

# LSTM Model
model_lstm = Sequential()
model_lstm.add(LSTM(units = 8, return_sequences = True, input_shape = (2500, 1), activation='tanh'))
model_lstm.add(LSTM(units = 8, return_sequences = True))
model_lstm.add(Dense(4, activation='tanh'))
model_lstm.add(Dropout(0.2))
model_lstm.add(Flatten())

# Combine CNN and LSTM model
#nb_classes = len(class_names)
nb = 7
combined = concatenate([model_CNN.output, model_lstm.output], axis=-1)
output = Dense(nb, activation='softmax')(combined)
model_final = Model(inputs=[model_CNN.input, model_lstm.input], outputs=output)

# Plot and compile the model
plot_model(model_final, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

model_final.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks
csv_logger = CSVLogger('training.log', separator=',', append=False)
mc = ModelCheckpoint('CNN_LSTM.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

# Training
time1 = time.time()
history = model_final.fit([train_images_cnn, train_images_lstm], 
                          train_labels_lstm,
                          batch_size=1000,
                          epochs=1, 
                          validation_data=([test_images_cnn, test_images_lstm],
                                           test_labels_lstm), 
                          callbacks=[mc, csv_logger])
print (("Training time=", time.time()-time1))

# Save training history
np.save("CNN_LSTM_history.npy", history.history)
 -->

# Model Analytics


In [22]:
from keras.models import load_model
fashion_model = load_model('./CNN_LSTM.h5') # load model
fashion_model.summary() # summarize model.

from contextlib import redirect_stdout
with open('./CNN_LSTM'+".xls", 'w') as f:
    with redirect_stdout(f):
        fashion_model.summary()
        
val_loss, val_accuracy=fashion_model.evaluate([test_images_cnn, test_images_lstm] ,test_labels_cnn) ## to get test accuracy and losses
print(val_loss, val_accuracy)


OSError: No file or directory found at ./CNN_LSTM.h5

In [ ]:
time2=time.time()
predict_prob=fashion_model.predict([test_images_cnn, test_images_lstm])
y_pred=np.argmax(predict_prob,axis=1)
print ('classification time:', time.time()-time2)

##print (y_pred)
y_true=np.argmax(test_labels_cnn, axis=1)
from sklearn.metrics import precision_recall_fscore_support as score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_true, y_pred)
print (cm)
print(classification_report(y_true, y_pred))

precision = precision_score(y_true, y_pred, average='weighted')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true, y_pred, average='weighted')
print('Recall: %f' % recall)
# f1: tp / (tp + fp + fn)
f1 = f1_score(y_true, y_pred, average='weighted')
print('F1 score: %f' % f1)
#-----------  IoU
from sklearn.metrics import jaccard_score
print ('IoU:', jaccard_score(y_true, y_pred, average='micro'))


test_eval = fashion_model.evaluate([test_images_cnn, test_images_lstm], test_labels_cnn)

loss, accuracy = fashion_model.evaluate([train_images_cnn, train_images_lstm], train_labels_cnn)
print('loss_train: ', loss, 'accuracy_train: ', accuracy)
print('Test loss:', test_eval[0], 'Test accuracy:', test_eval[1])

In [ ]:
import matplotlib.pyplot as plt

history_dict=history.history
loss_values=history_dict['loss']
val_loss_values=history_dict['val_loss']
acc_values=history_dict['accuracy']
val_acc_values=history_dict['val_accuracy']
epochs=range(1, len(acc_values)+1)
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points
loss_values=smooth_curve(loss_values)
val_loss_values=smooth_curve(val_loss_values)
acc_values=smooth_curve(acc_values)
val_acc_values=smooth_curve(val_acc_values)

font = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'normal',
        'size'   : 12}
        

plt.plot(epochs, acc_values, 'r-', label='Training acc')
plt.plot(epochs, val_acc_values, 'g', label='Validation acc')
plt.title('Training and Validation acc', fontdict=font)
plt.xlabel('Epochs', fontdict=font)
plt.ylabel('Accuracy', fontdict=font)
plt.legend()
plt.savefig("accuracy"+'CNN_LSTM'+".png")
plt.show()

plt.plot(epochs, loss_values, 'b-', label='Training loss')
plt.plot(epochs, val_loss_values, 'r', label='Validation loss')
plt.title('Training and Validation loss', fontdict=font)
plt.xlabel('Epochs',fontdict=font)
plt.ylabel('Loss',fontdict=font)
plt.legend()
plt.savefig("loss"+'CNN_LSTM'+".png")
plt.show()


In [ ]:
class_names = ['Fighting', 'Shoplifting', 'Abuse', 'Arrest', 'Shooting', 'Robbery', 'Explosion','RoadAccidents']
# train_dir = '/kaggle/input/ucf-crime-dataset/Train'
# class_names = os.listdir(train_dir)


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
#    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

font = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'normal',
        'size'   : 14}

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.savefig('confusion matrix1'+'CNN_LSTM'+'.png')
plt.show()
# Plot normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
plt.savefig('confusion matrix2'+'CNN_LSTM'+'.png')
plt.show()

# Testing with Input Image

In [ ]:
# import cv2
# import numpy as np
# from keras.models import load_model

# categories_labels = {'Fighting': 0, 'Shoplifting': 1, 'Abuse': 2, 'Arrest': 3, 'Shooting': 4, 'Robbery': 5, 'Explosion': 6}
# labels_categories = {v: k for k, v in categories_labels.items()}  # reverse dictionary for label lookup

# # Load the trained model
# model = load_model('/kaggle/input/cnn_lstm/keras/cnn_lstm/1/CNN_LSTM.h5')

# def predict_image(image_path):
#     # Load the image
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

#     # Resize the image
#     image = cv2.resize(image, (50, 50))

#     # Reshape the image to 4D array for CNN and LSTM input
#     image_cnn = image.reshape((1,) + image.shape + (1,))
#     image_lstm = image.reshape((1,) + (-1, 1))

#     # Use the model to predict the category of the image
#     prediction = model.predict([image_cnn, image_lstm])
    
#     # Find the category with the highest probability
#     label = np.argmax(prediction)

#     # Return the name of the category
#     return labels_categories[label]

# # Test the function
# image_path1 = "/kaggle/input/test-image/75089289-12483959-Kirk_Wharton_is_seen_on_CCTV_stashing_item_in_his_coat_in_a_foil-a-45_1693940026231.jpg"  # Replace with your image path
# img_path2="/kaggle/input/normal/Albisola-Marina-Italy.jpeg";
# i1="/kaggle/input/fighting/fight.jpg";
# i2="/kaggle/input/explosion/rawImage.jpg";
# i3="/kaggle/input/fight/fighting-1200x801.jpg";
# i4="/kaggle/input/robbery/r10_0_709_394_w1200_h678_fmax.jpg";
# i5="/kaggle/input/fight-5/cory-booker.jpg";
# #print(predict_image(image_path1));
# # print(predict_image(i2));
# print(predict_image(i3));
# # print("rob",predict_image(i4));
# # print(predict_image(i5));
# #print(predict_image(img_path2))

# def extract_and_predict(video_path):
#     # Open the video
#     cap = cv2.VideoCapture(video_path)

#     # Create a new directory to save predicted images
#     new_dir = 'predicted_images'
#     os.makedirs(new_dir, exist_ok=True)

#     # Initialize variables
#     frame_count = 0
#     last_category = None

#     # Loop through the frames
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Convert the frame to grayscale
#         frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#         # Predict the category of the frame
#         category = predict_image(frame_gray)

#         # Print the category if it's different from the last one
#         if category != last_category:
#             print('Frame', frame_count, 'Category:', category)
#         last_category = category

#         # Save the frame with the predicted category
#         cv2.imwrite(os.path.join(new_dir, f'{category}_{frame_count}.png'), frame_gray)

#         frame_count += 1

#     cap.release()
#     cv2.destroyAllWindows()

# # Test the function
# # video_path = "/kaggle/input/new-video/sample.mp4"  # Replace with your video path
# # extract_and_predict(video_path)

In [ ]:
# import smtplib
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# from email.mime.image import MIMEImage
# import getpass

# HOST = 'smtp-mail.outlook.com'
# PORT = 587

# FROM_EMAIL = "anomalytest@outlook.com"
# TO_EMAIL = "subashscr7@gmail.com"
# PASSWORD = "project24"

# predicted_category = predict_image(i2)
# MESSAGE = MIMEMultipart()
# MESSAGE['Subject'] = "Predicted Category"
# MESSAGE['From'] = FROM_EMAIL
# MESSAGE['To'] = TO_EMAIL

# # Attach predicted category as text
# text = MIMEText(f'The predicted category of the image is: {predicted_category}')
# MESSAGE.attach(text)

# # Attach image
# with open('/kaggle/input/robbery/r10_0_709_394_w1200_h678_fmax.jpg', 'rb') as img_file:
#     img = MIMEImage(img_file.read())
#     img.add_header('Content-Disposition', 'attachment', filename="r10_0_709_394_w1200_h678_fmax.jpg")
#     MESSAGE.attach(img)

# smtp = smtplib.SMTP(HOST, PORT)
# smtp.starttls()
# smtp.login(FROM_EMAIL, PASSWORD)

# smtp.sendmail(FROM_EMAIL, TO_EMAIL, MESSAGE.as_string())
# smtp.quit()


# TEST WITH RECORDED VIDEO

In [ ]:
# import cv2
# import numpy as np
# import os  # Import the os module
# from keras.models import load_model

# # Define the labels and categories
# categories_labels = {'Fighting': 0, 'Shoplifting': 1, 'Abuse': 2, 'Arrest': 3, 'Shooting': 4, 'Robbery': 5, 'Explosion': 6}
# labels_categories = {v: k for k, v in categories_labels.items()}  # reverse dictionary for label lookup

# # Load the trained model
# model = load_model('/kaggle/input/cnn_lstm/keras/cnn_lstm/1/CNN_LSTM.h5')

# def predict_image(image):
#     # Resize the image
#     image = cv2.resize(image, (50, 50))

#     # Reshape the image to 4D array for CNN and LSTM input
#     image_cnn = image.reshape((1,) + image.shape + (1,))
#     image_lstm = image.reshape((1,) + (-1, 1))

#     # Use the model to predict the category of the image
#     prediction = model.predict([image_cnn, image_lstm])

#     # Find the category with the highest probability
#     label = np.argmax(prediction)

#     # Return the name of the category
#     return labels_categories[label]

# def extract_and_predict(video_path):
#     # Open the video
#     cap = cv2.VideoCapture(video_path)

#     # Create a new directory to save predicted images
#     new_dir = 'predicted_images'
#     os.makedirs(new_dir, exist_ok=True)

#     # Initialize variables
#     frame_count = 0
#     last_category = None

#     # Loop through the frames
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Convert the frame to grayscale
#         frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#         # Predict the category of the frame
#         category = predict_image(frame_gray)

#         # Print the category if it's different from the last one
#         if category != last_category:
#             print('Frame', frame_count, 'Category:', category)
#         last_category = category

#         # Save the frame with the predicted category
#         cv2.imwrite(os.path.join(new_dir, f'{category}_{frame_count}.png'), frame_gray)

#         frame_count += 1

#     cap.release()
#     cv2.destroyAllWindows()

# # Test the function
# video_path = "/kaggle/input/test-video/WhatsApp Video 2024-02-12 at 21.58.53_abdb8c47.mp4"  # Replace with your video path
# extract_and_predict(video_path)


# TEST WITH GIF


In [ ]:
# import cv2
# import numpy as np
# import os
# from keras.models import load_model
# from PIL import Image

# categories_labels = {'Fighting': 0, 'Shoplifting': 1, 'Abuse': 2, 'Arrest': 3, 'Shooting': 4, 'Robbery': 5, 'Explosion': 6}
# labels_categories = {v: k for k, v in categories_labels.items()}  # reverse dictionary for label lookup

# # Load the trained model
# model = load_model('/kaggle/input/cnn_lstm/keras/cnn_lstm/1/CNN_LSTM.h5')

# def predict_image(image):
#     # Resize the image
#     image = cv2.resize(image, (50, 50))

#     # Reshape the image to 4D array for CNN and LSTM input
#     image_cnn = image.reshape((1,) + image.shape + (1,))
#     image_lstm = image.reshape((1,) + (-1, 1))

#     # Use the model to predict the category of the image
#     prediction = model.predict([image_cnn, image_lstm])

#     # Find the category with the highest probability
#     label = np.argmax(prediction)

#     # Return the name of the category
#     return labels_categories[label]

# def extract_and_predict(gif_path):
#     # Open the gif image
#     gif = Image.open(gif_path)

#     # Extract all frames from the gif
#     frames = []
#     try:
#         while True:
#             gif.seek(gif.tell() + 1)
#             frames.append(np.array(gif.convert('L')))  # Convert image to grayscale
#     except EOFError:
#         pass  # end of sequence

#     # Predict the category of each frame and save it in a new directory
#     new_dir = 'predicted_images'
#     os.makedirs(new_dir, exist_ok=True)
#     last_category = None  # variable to keep track of the last predicted category
#     for i, frame in enumerate(frames):
#         category = predict_image(frame)
#         if category != last_category:  # only print the category if it's different from the last one
#             print('Frame', i, 'Category:', category)
#         last_category = category  # update the last predicted category
#         cv2.imwrite(os.path.join(new_dir, f'{category}_{i}.png'), frame)

# # Test the function
# gif_path = "/kaggle/input/test-gif/WhatsApp Video 2024-02-12 at 21.51.27_95c67b1a.gif"  # Replace with your gif path
# extract_and_predict(gif_path)

# Sending mail through SMTP Server 

In [ ]:
# import smtplib
# from email.message import EmailMessage

# email = EmailMessage()
# email['from']='Subash'
# email['to']='subashs10011@gmail.com'
# email['subject']='You won 1,000,000 dollars !'

# email.set_content('I am a python master !')

# with smtplib.SMPTP(host='smtp.gmail.com', port=587) as smtp:
#     smtp.ehlo()
#     smtp.starttls()
#     smtp.login('dummyemail@gmail.com','abc123')
#     smtp.send_message(email)
#     print("email sent")
    

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

HOST = "smtp.outlook.com"
PORT = 587

FROM_EMAIL = "subashs12344@outlook.com"
TO_EMAIL = "subashs10011@gmail.com"
PASSWORD = "Project@12"

message = MIMEMultipart()
message['From'] = FROM_EMAIL
message['To'] = TO_EMAIL
message['Subject'] = "Hello, this is Subash"

# Add message body
body = "This is the message body."
message.attach(MIMEText(body, 'plain'))

# Attach image
image_url = "/kaggle/input/fight/fighting-1200x801.jpg"  # Update the image URL with the correct path
with open(image_url, 'rb') as attachment:
    image_part = MIMEImage(attachment.read(), name="fighting-1200x801.jpg")

image_part.add_header('Content-Disposition', 'attachment', filename="fighting-1200x801.jpg")
message.attach(image_part)


In [ ]:
import smtplib
import getpass

HOST = 'smtp-mail.outlook.com'
PORT = 587

FROM_EMAIL = "subash12344@outlook.com"
TO_EMAIL = "subashscr7@gmail.com"
PASSWORD = "Project@12"

MESSAGE = """Subject: hello this is subash"""

smtp = smtplib.SMTP(HOST, PORT)

status_code, response = smtp.ehlo()
print(f"[*] Echoing the server: {status_code} {response}")

status_code, response = smtp.starttls()
print(f"[*] Starting TLS connection: {status_code} {response}")

status_code, response = smtp.login(FROM_EMAIL, PASSWORD)
print(f"[*] Logging in: {status_code} {response}")

smtp.sendmail(FROM_EMAIL, TO_EMAIL, MESSAGE)
smtp.quit()

In [ ]:
import smtplib
import getpass

HOST = "smtp-mail.outlook.com"
PORT = 587

FROM_EMAIL = "subash12344@outlook.com"
TO_EMAIL = "subashscr7@gmail.com"
PASSWORD = "Project@12"

MESSAGE = """Subject: hello this is subash"""

smtp = smtplib.SMTP(HOST, PORT)

status_code, response = smtp.ehlo()
print(f"[*] Echoing the server: {status_code} {response}")

status_code, response = smtp.starttls()
print(f"[*] Starting TLS connection: {status_code} {response}")

status_code, response = smtp.login(FROM_EMAIL, PASSWORD)
print(f"[*] Logging in: {status_code} {response}")

smtp.sendmail(FROM_EMAIL, TO_EMAIL, MESSAGE)
smtp.quit()

TEST EMAIL KSK

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

HOST = 'smtp-mail.outlook.com'
PORT = 587
USERNAME = 'subash12344@outlook.com'
PASSWORD = 'Project@12'

# Create a MIME multipart message
message = MIMEMultipart()
message['From'] = USERNAME
message['To'] = 'sureshvel2002@gmail.com'
message['Subject'] = 'Test Email'

# Add message body
message.attach(MIMEText('This is a test email.', 'plain'))

# Connect to the SMTP server
try:
    smtp = smtplib.SMTP(HOST, PORT)
    smtp.starttls()
    smtp.login(USERNAME, PASSWORD)

    # Send the email
    smtp.sendmail(USERNAME,'sureshvel2002@gmail.com', message.as_string())
    print('Email sent successfully!')

except Exception as e:
    print('Error sending email:', e)

finally:
    if 'smtp' in locals():
        smtp.quit()


In [ ]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

def send_message_with_image(message, image_path):
    smtp_server = 'smtp-mail.outlook.com'  # Update with your SMTP server
    smtp_port = 587  # Update with your SMTP port number

    # Connect to the SMTP server
    s = smtplib.SMTP(smtp_server, smtp_port)
    s.starttls()  # Enable TLS encryption
    # If your SMTP server requires authentication, uncomment the following lines and provide your credentials
    s.login('subash12344@outlook.com','Project@12')

    # Attach image to the message
    with open(image_path, 'rb') as fp:
        img_data = fp.read()
    image = MIMEImage(img_data, name=os.path.basename(image_path))
    message.attach(image)

    # Send the email
    s.sendmail(message['From'], message['To'], message.as_string())

    # Close the connection
    s.quit()

def create_message(sender, recipient, subject, body):
    # Create the email message
    message = MIMEMultipart()
    message['From'] = sender
    message['To'] = recipient
    message['Subject'] = subject

    # Attach the body of the email
    message.attach(MIMEText(body, 'plain'))

    return message

if __name__ == '__main__':
    # Provide the necessary details
    sender_email = 'subash12344@outlook.com'
    recipient_email = 'sureshvel2002@gmail.com'
    subject = 'Message with Image'
    body = 'This email contains an image attachment.'

    # Create the message
    message = create_message(sender_email, recipient_email, subject, body)

    # Provide the path to the image file
    image_path = '/kaggle/input/fight/fighting-1200x801.jpg'  # Update with the correct image path

    # Send the message with the image attachment
    send_message_with_image(message, image_path)